### Import your librares

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib as plt 
import seaborn as sns

### Load your dataset

In [ ]:
df = pd.read_csv("seattle_weather.csv")
df

### Handle missing data in PRCP and RAIN columns

In [ ]:
df.hist()

In [ ]:
df.dtypes

In [ ]:
df[df['PRCP'].isnull()]

In [ ]:
df.var()

In [ ]:
# filled with False and the variance cahnge is not noticable 
df['RAIN'].fillna(False, inplace=True) 

# filled with 0.0 float and the variance cahnge is not noticable 
df['PRCP'].fillna(0.0,inplace=True)

In [ ]:
df.var()

### Repeat the following steps for each column within outliers

* Visualize the outliers
* Detect if your data set contains any outliers
* Identify lower & higher limit of outliers
* Drop all outliers
* Visualize the column after processing


In [ ]:
df.describe()

In [ ]:
sns.boxplot(data=df)

# Handling TMAX column 

In [ ]:
Q1 = np.percentile(df['TMAX'], 25, interpolation = 'midpoint')
Q3 = np.percentile(df['TMAX'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1
               
print('lower bound outliers ', Q1 - 1.5*(IQR))
print('higher bound outliers', Q3 + 1.5*(IQR))

In [ ]:
df = df.drop(df[(df['TMAX'] > 97.5) | (df['TMAX'] < 21.5)].index)

# Handling PRCP

In [ ]:
Q1 = np.percentile(df['PRCP'], 25, interpolation = 'midpoint')
Q3 = np.percentile(df['PRCP'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1
               
print('lower bound outliers ', Q1 - 1.5*(IQR))
print('higher bound outliers', Q3 + 1.5*(IQR))

In [ ]:
df = df.drop(df[(df['PRCP'] > 0.25) | (df['PRCP'] < -0.15000000000000002)].index)

# Handling TMIN 

In [ ]:
Q1 = np.percentile(df['TMIN'], 25, interpolation = 'midpoint')
Q3 = np.percentile(df['TMIN'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1
               
print('lower bound outliers ', Q1 - 1.5*(IQR))
print('higher bound outliers', Q3 + 1.5*(IQR))

In [ ]:
df = df.drop(df[(df['TMIN'] > 73.0) | (df['TMIN'] < 17.0)].index)

# Display after removing outliers

In [ ]:
sns.boxplot(data=df)

In [ ]:
df.var()

In [ ]:
df.describe()